In [1]:
import os
import sys
import glob

try:
    sys.path.append(glob.glob('carla/PythonAPI/carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

import math
import time
import hydra
import logging
import argparse

from numpy import random
from omegaconf import DictConfig

from utils.weather import Weather
from utils.vehicle_manager import VehicleManager
from utils.pedestrian_manager import PedestrianManager
from utils.common import get_actor_blueprints

In [2]:
def main(cfg: DictConfig):
    logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)
    
    client = carla.Client(cfg.host, cfg.port)
    client.set_timeout(10.0)

    vcfg = hydra.compose(config_name="vehicle_manager.yaml")
    print(vcfg)
    tm = VehicleManager(client=client, cfg=vcfg)

    pcfg = hydra.compose(config_name="pedestrian_manager.yaml")
    print(pcfg)
    pm = PedestrianManager(client=client, cfg=pcfg)
    try:
        world = client.get_world()

        tm.spawn_vehicles(number_of_vehicles=30)
        pm.spawn_walkers(number_of_walkers=pcfg.number_of_walkers)

        # Dynamic Weather
        speed_factor = 1.0
        update_freq = 0.5 / speed_factor

        weather = Weather(world.get_weather())

        elapsed_time = 0.0

        synchronous_master = world.get_settings().synchronous_mode
        while True:
            if not cfg.asynch and synchronous_master:
                world.tick()
            else:
                timestamp = world.wait_for_tick(seconds=30.0).timestamp
                elapsed_time += timestamp.delta_seconds
                if elapsed_time > update_freq:
                    weather.tick(speed_factor * elapsed_time)

                    world.set_weather(weather.weather)
                    sys.stdout.write('\r' + str(weather) + 12 * ' ')
                    sys.stdout.flush()
                    elapsed_time = 0.0

    finally:

        tm.destroy()
        pm.destroy()
        time.sleep(0.5)

In [3]:
try:
    with hydra.initialize(version_base=None, config_path="conf"):
        cfg = hydra.compose(config_name="config.yaml")
        print(cfg)
        main(cfg)
except KeyboardInterrupt:
    pass
finally:
    print('\ndone.')

{'host': '127.0.0.1', 'port': 2000, 'asynch': True}
{'host': '127.0.0.1', 'port': 2000, 'asynch': True, 'tm_port': 8000, 'safe': True, 'filterv': 'vehicle.*', 'generationv': 'All', 'hybrid': False, 'seed': 13, 'car_lights_on': False, 'hero': False, 'respawn': False, 'no_rendering': False}
You are currently in asynchronous mode. If this is a traffic simulation,             you could experience some issues. If it's not working correctly, switch to synchronous             mode by using self.traffic_manager.set_synchronous_mode(True)
{'host': '127.0.0.1', 'port': 2000, 'asynch': True, 'number_of_walkers': 10, 'filterw': 'walker.pedestrian.*', 'generationw': '2', 'seedw': 0, 'running_percentage': 0.0, 'crossing_percentage': 0.0}

Added 30 vehicles
Spawned 10 walkers.
Sun(alt: -14.84, azm: 322.45) Storm(clouds=1%, rain=0%, wind=5%)            
Destroying 30 vehicles
Destroying 10 walkers

done.
